In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load and preprocess data 

In [3]:
df_train = pd.read_csv('train.csv', index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)

In [4]:
# identify and replace missing values
print(df_train.isnull().sum())
print(df_test.isnull().sum())

print(df_train['opened_position_qty '].mean())
print(df_train['opened_position_qty '].median())

print(df_train['closed_position_qty'].mean())
print(df_train['closed_position_qty'].median())

# replace missing values with median (less sensitive to outliers)
df_train['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_test['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_train['closed_position_qty'].fillna(df_train['closed_position_qty'].median(),inplace=True)
df_test['closed_position_qty'].fillna(df_train['closed_position_qty'].median(), inplace=True)

last_price                   0
mid                          0
opened_position_qty     172460
closed_position_qty     172460
transacted_qty               0
d_open_interest              0
bid1                         0
bid2                         0
bid3                         0
bid4                         0
bid5                         0
ask1                         0
ask2                         0
ask3                         0
ask4                         0
ask5                         0
bid1vol                      0
bid2vol                      0
bid3vol                      0
bid4vol                      0
bid5vol                      0
ask1vol                      0
ask2vol                      0
ask3vol                      0
ask4vol                      0
ask5vol                      0
y                            0
dtype: int64
last_price                  0
mid                         0
opened_position_qty     53656
closed_position_qty     53656
transacted_qty              0


In [5]:
# Normalize data
from sklearn import preprocessing

x_train = df_train[df_train.columns[:26]]
y_train = df_train['y']
x_test = df_test

# Normalize training data by subtracting mean and scaling to unit variance
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
x_train = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns)

# Normalize testing data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
x_test = pd.DataFrame(x_test_norm, index=x_test.index, columns=x_test.columns) 

## Build and train model

In [6]:
model = keras.Sequential([
 keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(x_train.shape[1],)),
 keras.layers.Dense(128, activation=tf.nn.relu),
 keras.layers.Dense(256, activation=tf.nn.relu),
 keras.layers.Dense(512, activation=tf.nn.relu),
 keras.layers.Dense(2, activation=  'softmax')
 ])

In [7]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_model = model.fit(
 x_train, y_train,
 epochs= 30, batch_size = 128)

Epoch 1/30
592380/592380 [==============================] - 25s 42us/step - loss: 0.6276 - acc: 0.6481
Epoch 2/30
592380/592380 [==============================] - 30s 50us/step - loss: 0.6251 - acc: 0.6501
Epoch 3/30
592380/592380 [==============================] - 29s 48us/step - loss: 0.6244 - acc: 0.6515
Epoch 4/30
592380/592380 [==============================] - 31s 52us/step - loss: 0.6240 - acc: 0.6517
Epoch 5/30
592380/592380 [==============================] - 31s 52us/step - loss: 0.6237 - acc: 0.6521
Epoch 6/30
592380/592380 [==============================] - 29s 50us/step - loss: 0.6236 - acc: 0.6521
Epoch 7/30
592380/592380 [==============================] - 27s 45us/step - loss: 0.6233 - acc: 0.6526
Epoch 8/30
592380/592380 [==============================] - 27s 45us/step - loss: 0.6231 - acc: 0.6529
Epoch 9/30
592380/592380 [==============================] - 26s 43us/step - loss: 0.6229 - acc: 0.6528
Epoch 10/30
592380/592380 [==============================] - 29s 48us/ste

## Save results

In [ ]:
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[['Predicted']].to_csv('submission.csv')